

Воспользовавшись предобученной моделью [ResNet](https://pytorch.org/hub/pytorch_vision_resnet/), мы построим классификатор изображений с кошками и собаками на датасете [KaggleCatsAndDogs](https://www.microsoft.com/en-us/download/details.aspx?id=54765).

Модель ResNet построена на свёртках. Эта матричная операция занимает много времени, будучи выполняемой на CPU. Архитектура CUDA позволяет существенно ускорить вычисление матричных операций благодаря параллельным вычислениям и использованию графических процессоров. Именно поэтому с этим ноутбуком лучше работать в Google Colab, так как он даёт возможность работы с видеокартой.

### 1. Готовимся к работе.
- Подключаем Google Drive(где должен лежать датасет скачанный KaggleCatsAndDogs).
- Устанавливаем и импортируем необходимые библиотеки.
- Извлекаем данные из архива.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
ls

 CDLA-Permissive-2.0.pdf   drive/   PetImages/  'readme[1].txt'   sample_data/


In [25]:
#!pip install torchinfo
#!unzip "/content/drive/MyDrive/Skillbox/ML Advanced/kagglecatsanddogs_5340.zip"

In [4]:
import os
from glob import glob
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import ToTensor, Compose, Resize, Normalize, ToPILImage
from PIL import Image
from torchinfo import summary
from torchvision import models

### 2. Готовим датасет.
- Описываем класс кастомного датасета, наследуясь от `torch.utils.data.Dataset`
- Загружаем датасет и делим его на тренировочную и валидационную выборки.
- Оборачиваем датасеты в DataLoader.

нормируем и центрируем наши данные не на (0.5, 0.5), а на заранее вычисленные среднее и корень из дисперсии изображений из датасета ImageNET. Так мы приближаем наши данные к тому, с чем училась работать модель ResNet. По этой же причине мы приводим все данные к размеру 224 × 224 пикселя.

In [10]:
ls

 CDLA-Permissive-2.0.pdf   drive/   PetImages/  'readme[1].txt'   sample_data/


In [11]:
class CatsDogsDataset(Dataset):
    def __init__(self, datapath, transform=None):
        super(CatsDogsDataset, self).__init__()
        self.paths = []  # пути до картинок
        self.labels = [] # labels картинок
        # кошкам сделали соответствие - 0, собакам -1
        for y, cls in enumerate(['Cat', 'Dog']): 
            p_tmp = glob(os.path.join(datapath, cls, '*.jpg')) # для каждого класса создаем список изображений
            l_tmp = [y] * len(p_tmp)                           # для каждого класса создаем список labels
            # добавляем списки к self.paths и self.labels
            self.paths.extend(p_tmp)
            self.labels.extend(l_tmp)
            # добавляем transform
        self.transform = transform

    def __len__(self):
        return len(self.paths) # длина датасета это длина листа с путями

    def __getitem__(self, idx): # метод __getitem__ принимает индексы элементов в датасете и возвращает изображения и label
        img = Image.open(self.paths[idx]) # считываем изображение
        img = img.convert('RGB')          # !!! загружаем изображения с одним каналом !!!
        label = self.labels[idx]          # сохраняем label

        if self.transform is not None:    # применяем transform
            img = self.transform(img)

        return img, label

In [12]:
dataset = CatsDogsDataset(datapath='/content/PetImages')

In [13]:
print("Длина датасета:", len(dataset))

Длина датасета: 25000


In [1]:
dataset = CatsDogsDataset('/content/PetImages')
for i in range(10):
    display(dataset[i] [0]) # так смотрим картинку
    display(dataset[-i] [0])
# а так смотрим пару - img, label
for i in range(10):
    display(dataset[i])
    display(dataset[-i])

# for i in range(10):
#     display(dataset[i % len(dataset)])

In [10]:
# from torchvision.transforms import Lambda
# import torch.nn.functional as F

In [15]:
# делаем тензор
transform = Compose([
    ToTensor(),
    Resize((224, 224)), # установим размер
    Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # нормализуем - среднее и корень из дисперсии...
])

dataset = CatsDogsDataset('/content/PetImages', transform=transform)

random_generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset = random_split(dataset, [0.8, 0.2], generator=random_generator)
print(len(train_dataset), len(val_dataset))

train_loader = DataLoader(train_dataset, batch_size=200, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1000)

# #распечатать форму и тип данных тензора изображений в начале цикла обучения
# for step, (data, target) in enumerate(train_loader):
#     data_transformed = transform(data)
#     print(f'Data shape: {data_transformed.shape}, Data type: {data_transformed.dtype}')

20000 5000


### 3. Собираем модель.
- Скачиваем веса обученной модели ResNet, например resnet-50.
- Замораживаем параметры модели путём отключения для них процесса вычисления градиентов и обновления весов.
- Меняем последний слой под задачу бинарной классификации.

Для задачи бинарной классификации мы выберем бинарную кросс-энтропию в качестве лосса — [BCELoss](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html). В отличие от CrossEntropyLoss, BCELoss ожидает на вход именно вероятность принадлежности семпла к целевому классу. Именно поэтому нам обязательно нужно применить сигмоиду к выходу линейного слоя.

In [16]:
resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
summary(resnet, input_size=(1, 3, 224, 224)) # архитектура свертки...

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 181MB/s]


Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 1000]                 --
├─Conv2d: 1-1                            [1, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         128
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 256, 56, 56]          --
│    └─Bottleneck: 2-1                   [1, 256, 56, 56]          --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           4,096
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]           --
│ 

In [17]:
# итог предыдущей свертки - ветор Linear: 1-10 [1, 1000] 1000 элементов (классов)
# а нам нужно ДВА класса - кот собака
# заменяем последний слой Linear: 1-10
for param in resnet.parameters():
    param.requires_grad = False #  предиктор учить уже не будем(считать градиеты и обновлять веса ), оставим как есть...

fc_inputs = resnet.fc.in_features # в слое AdaptiveAvgPool2d перед Linear 2048 входных фичей
resnet.fc = nn.Sequential(        # заменяем на наш линейный слой
    nn.Linear(fc_inputs, 1),      # тут вероятность, т.е. число от 0 до 1
    nn.Sigmoid()                  # в наш слой добавили сигмоиду чтобы на выходе было от 0 до 1 (бинарная классификация)
)

### 4. Обучаем модель и сохраняем веса.
Пишем функции `train()` и `validate()`. Здесь добавится новый аргумент функций — `device`, так как мы хотим работать с моделью на GPU, но должны иметь возможность запустить её также и на CPU.

In [18]:
def train(model, optimizer, loss_f, train_loader, val_loader, n_epoch, val_fre, device): # тут + параметр device
    model = model.to(device)
    model.train()
    for epoch in range(n_epoch):
        loss_sum = 0
        print(f'Epoch: {epoch}')
        for step, (data, target) in enumerate(train_loader):
            target = target.to(torch.float32)
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data).squeeze(1)
            loss = loss_f(output, target)
            loss.backward()
            optimizer.step()

            loss_sum += loss.item()

            if step % 10 == 0:
                print(f'Iter: {step} \tLoss: {loss.item()}')

        print(f'Mean Train Loss: {loss_sum / (step + 1):.6f}', end='\n\n')

        if epoch % val_fre == 0:
            validate(model, val_loader, device)

def validate(model, val_loader, device):
    model = model.to(device)
    model.eval()
    loss_sum = 0
    correct = 0
    for step, (data, target) in enumerate(val_loader):
        target = target.to(torch.float32)
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            output = model(data).squeeze(1)
            loss = loss_f(output, target)
        loss_sum += loss.item()
        pred = torch.round(output)
        correct += pred.eq(target.view_as(pred)).sum().item()
    acc = correct / len(val_loader.dataset)
    print(f'Val Loss: {loss_sum / (step + 1):.6f} \tAccuracy: {acc}')
    model.train()

In [21]:
loss_f = nn.BCELoss()
optimizer = torch.optim.SGD(resnet.parameters(), lr=1e-3)

n_epoch = 10
val_fre = 1

device = 'cuda'
#device = 'cpu'

train(resnet, optimizer, loss_f, train_loader, val_loader, n_epoch, val_fre, device)

validate(resnet, val_loader, device)

Epoch: 0
Iter: 0 	Loss: 0.6926815509796143
Iter: 10 	Loss: 0.6715524792671204
Iter: 20 	Loss: 0.6633674502372742
Iter: 30 	Loss: 0.6525170207023621
Iter: 40 	Loss: 0.627971351146698
Iter: 50 	Loss: 0.611298143863678
Iter: 60 	Loss: 0.5973123908042908
Iter: 70 	Loss: 0.5961748957633972


/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Iter: 80 	Loss: 0.581307053565979
Iter: 90 	Loss: 0.5638806819915771
Mean Train Loss: 0.620319

Val Loss: 0.556834 	Accuracy: 0.8932
Epoch: 1
Iter: 0 	Loss: 0.5562268495559692
Iter: 10 	Loss: 0.5475522875785828
Iter: 20 	Loss: 0.5416414737701416
Iter: 30 	Loss: 0.5287393927574158
Iter: 40 	Loss: 0.516267716884613
Iter: 50 	Loss: 0.5108008980751038
Iter: 60 	Loss: 0.4997338056564331
Iter: 70 	Loss: 0.49052056670188904
Iter: 80 	Loss: 0.5015973448753357
Iter: 90 	Loss: 0.4736279249191284
Mean Train Loss: 0.510059

Val Loss: 0.465012 	Accuracy: 0.9494
Epoch: 2
Iter: 0 	Loss: 0.4776754677295685
Iter: 10 	Loss: 0.4720796048641205
Iter: 20 	Loss: 0.44427409768104553
Iter: 30 	Loss: 0.44800499081611633
Iter: 40 	Loss: 0.42262670397758484
Iter: 50 	Loss: 0.4334222972393036
Iter: 60 	Loss: 0.4309605360031128
Iter: 70 	Loss: 0.4094736874103546
Iter: 80 	Loss: 0.3900860846042633
Iter: 90 	Loss: 0.407488077878952
Mean Train Loss: 0.433157

Val Loss: 0.401925 	Accuracy: 0.964
Epoch: 3
Iter: 0 	Loss

In [47]:
#ls -l /content/PetImages/Cat/666.jpg # ошибка - нулевое содержимое, скопировал в него 665.jpg и т.д.

-rw-r--r-- 1 root root 14253 Oct 24 06:48 /content/PetImages/Cat/666.jpg


In [20]:
#cp /content/PetImages/Cat/665.jpg /content/PetImages/Cat/666.jpg

In [60]:
#ls -l /content/PetImages/Dog/11702.jpg

-rw-r--r-- 1 root root 6555 Oct 24 06:59 /content/PetImages/Dog/11702.jpg


In [19]:
#cp /content/PetImages/Dog/11701.jpg /content/PetImages/Dog/11702.jpg

In [22]:
torch.save(resnet.fc.state_dict(), '/content/drive/MyDrive/Skillbox/ML Advanced/resnet_fc.pth')

In [23]:
resnet.fc.load_state_dict(torch.load('/content/drive/MyDrive/Skillbox/ML Advanced/resnet_fc.pth'))

<ipython-input-23-7de92500e102>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet.fc.load_state_dict(torch.load('/content/drive/MyDrive/Skillbox/ML Advanced/resnet_fc.

<All keys matched successfully>